In [20]:
import openai
from dotenv import load_dotenv
import os
import requests
from datetime import datetime, time, timezone, timedelta
import random
from bs4 import BeautifulSoup
import json
import re

load_dotenv()
API_URL = os.getenv("API_URL")

In [21]:
class Saint:

    def __init__(self, day, month):
        self.url = f'https://www.vaticannews.va/pt/santo-do-dia/{"{:02}".format(month)}/{"{:02}".format(day)}.html'
        self.referenceDate = f'{"{:02}".format(day)}-{"{:02}".format(month)}'
        self.html = self.get_html()
        
        self.readingRef, self.reading = self.get_reading()




    def get_html(self):
        browsers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome / 86.0.4240.198Safari / 537.36"}
        html_content = requests.get(self.url, headers=browsers)
        soup = BeautifulSoup(html_content.text, 'html.parser')
        return soup
    
    
    
    def corrigir_texto(self, texto):
        # Remover quebras de linha no meio das frases, onde há vírgulas ou dois pontos
        texto_corrigido = re.sub(r'(\,\s*\n|\:\s*\n|\.\s*\n)', lambda m: f'{ m.group(0).strip()}', texto)
        # Juntar as linhas que não estão devidamente separadas (começando com letra minúscula)
        texto_corrigido = re.sub(r'\n([a-z])', r' \1', texto_corrigido)
        # Adicionar quebras de linha nos parágrafos corretos (onde há dois pontos, exclamação ou final de frase com ponto)
        texto_corrigido = re.sub(r'(\.\s+)', r' \1\n\n', texto_corrigido)
        return texto_corrigido.strip()



    def get_reading(self):
        blocks = self.html.find_all(class_='section section--evidence section--isStatic')
        reading = ''
        for b in blocks:
            saint = b.find('div', class_='section__head').get_text().strip()
            print(saint)
            vaticanNews_short = b.find('div', class_='section__content').find('p').get_text().strip()
            print(vaticanNews_short)
            try:
                vaticanNews_img = b.find('div', class_='section__content').find('img').get('src')
                print(vaticanNews_img)
            except:
                    vaticanNews_img = ''
            try:
                vaticanNews_long_url = 'https://www.vaticannews.va' + b.find('div', class_='section__content').find('a', class_='saintReadMore').get('href')
                print(vaticanNews_long_url)
                get_vaticanNews_long = self.get_vaticanNews_long(vaticanNews_long_url)
                print(get_vaticanNews_long)
            except:
                    vaticanNews_img = ''
                    get_vaticanNews_long = ''

        return 'reading_reference', 'reading'
    
    
    
    def get_vaticanNews_long(self, vaticanNews_long_url):
        browsers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome / 86.0.4240.198Safari / 537.36"}
        html_content = requests.get(vaticanNews_long_url, headers=browsers)
        soup = BeautifulSoup(html_content.text, 'html.parser')
        details = soup.find('div', class_='section__content santi--detail').findAll('p')
        result = ''
        for d in details:
            result += d.get_text().strip() + '\n'
        return result.strip()


    
    
    def save_saint(self):
        try:
            url = f'{API_URL}/api/prayer'
            url = f'http://localhost:3000/api/prayer'
            payload = {
                # "uid": "johannesadmin",
                # "supportUid": "johannesadmin",
                "reading": self.reading,
                "readingRef": self.readingRef,
                "gospel": self.gospel,
                "gospelRef": self.gospelRef,
                "popeWords": self.popeWords,
                "popeWordsRef": self.popeWordsRef,
                "referenceDate": self.referenceDate,
                "__t": "daily_saint",
            }
            response = requests.post(url, json=payload)
            if response.status_code != 200:
                print(f'Erro: {response.status_code}')
        except Exception as e:
            print("Problema ao salvar notícia:", e)



In [22]:
# Baseado nas datas disponíveis no calendário de https://www.vaticannews.va/pt/palavra-do-dia.html
year = 2024
month = 9

a = Saint(5,9)

# for day in range(1, 24):
#     try:
#         o = Oracao(day, month, year)
#         o.save_pray()
#     except Exception as e:
#         print(f'Erro no dia {day}/{month}/{year}:', e)


SS. Aconto, Nono, Herculano e Taurino, mártires de Porto Romano
Não há muitas notícias sobre estes mártires de Fiumicino, porto romano, a não ser nas Lendas de Santa Áurea. O Papa Formoso levou as relíquias de Nono, Herculano e Taurino para uma igreja na Ilha Tiberina, em Roma, dado que já havia outra igreja dedicada a Acôncio, segundo documentos do século X.
Santa Madre Teresa de Calcutá
Madre Teresa, testemunha da caridade e da misericórdia, é um exemplo para todos, não só cristãos, das maravilhas que uma pessoa,  pode realizar.
Nobel da Paz, fundadora das Missionárias da Caridade, a “Mãe dos pobres” – canonizada pelo Papa Francisco – é celebrada  no dia 5 de setembro.
data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7
https://www.vaticannews.va/pt/santo-do-dia/09/05/santa-madre-teresa-de-calcuta-.html
“Apresentou-lhes a mulher mais poderosa do mundo!” No dia 26 de outubro de 1985, o Secretário geral da ONU, Pérez de Cuéllar, assim apresentou Madre Teresa 